In [1]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import rasterio
from torchvision import transforms
import torchvision.models as models
from torchvision.io import read_image
from torchvision.transforms import ToTensor, Normalize
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data.dataset import Subset
from sklearn.metrics import classification_report
import os
import matplotlib.pyplot as plt
import random
import pdb
from collections import Counter
import segmentation_models_pytorch as smp
import albumentations as albu
from skimage.io import imread
import pandas as pd
import cv2
tqdm.pandas()

/work/scorreacardo_umass_edu/miniconda3/envs/urbano2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path_to_tiles = "/gypsum/eguide/projects/scorreacardo/urbano/data/ts_predict_sample/kenya/02356b68-aa6f-4e4c-8a42-d225b2b562d4/images"
path_to_masks = "/gypsum/eguide/projects/scorreacardo/urbano/data/ts_predict_sample/kenya/02356b68-aa6f-4e4c-8a42-d225b2b562d4/labels_mask"

In [13]:
# Define the root directory containing the satellite image tiles
root_dir_imgs = "/gypsum/eguide/projects/scorreacardo/urbano/data/ts_predict_sample/kenya/02356b68-aa6f-4e4c-8a42-d225b2b562d4/images"
root_dir_masks = "/gypsum/eguide/projects/scorreacardo/urbano/data/ts_predict_sample/kenya/02356b68-aa6f-4e4c-8a42-d225b2b562d4/labels_mask"
# Define the output directory for the chips
output_dir_imgs = "/work/scorreacardo_umass_edu/DeepSatGSD/data/processed/inference_ts/imgs"
output_dir_masks = "/work/scorreacardo_umass_edu/DeepSatGSD/data/processed/inference_ts/masks"
# Define the desired chip size
chip_size = 256

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir_imgs):
    os.makedirs(output_dir_imgs)
    
if not os.path.exists(output_dir_masks):
    os.makedirs(output_dir_masks)


# Initialize a list to store the chip filenames and dates
chip_filenames = []
chip_dates = []

for image_name in tqdm(os.listdir(root_dir_imgs)):

# # Walk through the root directory and its subdirectories
# for root, dirs, files in os.walk(root_dir):
    # Iterate over the files in the current directory
#for file in tqdm(files):
    # Check if the file is a PNG image
    if image_name.lower().endswith(".jpg"):
        # Get the full path of the image file
        image_path = os.path.join(root_dir_imgs, image_name)
        mask_path = os.path.join(root_dir_masks, image_name[:-4] + ".png")

        # Open the image file
        image = Image.open(image_path)
        mask = Image.open(mask_path)

        # Get the image size
        image_width, image_height = image.size

        # Iterate over the image in a sliding window fashion to extract chips
        for y in range(0, image_height - chip_size + 1, chip_size):
            for x in range(0, image_width - chip_size + 1, chip_size):
                # Extract the chip from the image
                chip_img = image.crop((x, y, x + chip_size, y + chip_size))
                chip_mask = mask.crop((x, y, x + chip_size, y + chip_size))

                # Get the date from the filename
                year = image_name.split("_")[0]
                month = image_name.split("_")[1]
                day = image_name.split("_")[2]
                tilename = image_name.split("_")[4]

                # Create a unique filename for the chip
                chip_img_filename = f"{year}_{month}_{day}_{tilename}_{x}_{y}.png"
                chip_mask_filename = f"{year}_{month}_{day}_{tilename}_{x}_{y}.png"
                

                # Save the chip
                chip_path_imgs = os.path.join(output_dir_imgs, chip_img_filename)
                chip_img.save(chip_path_imgs)
                
                # Save the chip
                chip_path_masks = os.path.join(output_dir_masks, chip_mask_filename)
                chip_mask.save(chip_path_masks)


100%|██████████| 4/4 [00:16<00:00,  4.05s/it]


## Pipeline

In [3]:
class_map = {0:'GSD_50cm',
 1:'GSD_65cm',
 2:'GSD_80cm',
 3:'GSD_100cm',
 4:'GSD_124cm',
 5:'GSD_150cm',
 6:'GSD_175cm',
 7:'GSD_200cm',
 8:'GSD_250cm',
 9:'GSD_300cm'}

In [4]:
def get_model(gsd, aug=False):
    if aug:
        exp_dic = {
            '50cm': 'exp2',
            '65cm': 'exp4', 
            '80cm': 'exp6'
        }
    else:
        exp_dic = {
            '50cm': 'exp1',
            '65cm': 'exp3', 
            '80cm': 'exp5'
        }
        
    model = torch.load(f'/work/scorreacardo_umass_edu/DeepSatGSD/models/segmentation_{exp_dic[gsd]}_{gsd}_best_model.pth')
    return model

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

def sigmoid(X):
    return 1/(1+np.exp(-X))

def iou(pr_mask, gt_mask):
    pr_mask_area = np.count_nonzero(pr_mask)
    gt_mask_area = np.count_nonzero(gt_mask)
    intersection = np.count_nonzero(np.logical_and(pr_mask, gt_mask))
    if pr_mask_area + gt_mask_area - intersection == 0:
        return None
    iou = intersection/(pr_mask_area + gt_mask_area - intersection)
    return iou

def se(pr_mask, gt_mask):
    pr_contours, _ = cv2.findContours(pr_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    gt_contours, _ = cv2.findContours(gt_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sq_error = (len(pr_contours) - len(gt_contours))**2
    return sq_error

def perc_error(pr_mask, gt_mask):
    pr_contours, _ = cv2.findContours(pr_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    gt_contours, _ = cv2.findContours(gt_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(gt_contours) == 0:
        return None
    perc_error = (len(pr_contours) - len(gt_contours))/len(gt_contours)
    return perc_error*100

def perc_area(pr_mask, gt_mask):
    #area as percentage of total area
    pr_pixels = cv2.countNonZero(pr_mask)
    gt_pixels = cv2.countNonZero(gt_mask)
    if gt_pixels == 0:
        return None
    return (pr_pixels/gt_pixels)*100

In [14]:
# Move the model to the appropriate device (e.g., GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Create an instance of the same model architecture
model = models.resnet18(pretrained=False)
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)
# Load the saved model state dictionary
model.load_state_dict(torch.load('/work/scorreacardo_umass_edu/DeepSatGSD/models/trained_model_gsd_resnet18_epochs_3_training_40_perc.pt'))
model.eval()  # Set the model to evaluation mode
# Define the transformation to be applied to the images
transform = torchvision.transforms.Compose([
    ToTensor(),
    Normalize([0.46619912981987, 0.4138015806674957, 0.2945951819419861], 
              [0.19115719199180603, 0.1479424238204956, 0.13974712789058685])  #
])

# Path to the folder containing the images for inference
inference_folder = '/work/scorreacardo_umass_edu/DeepSatGSD/data/processed/inference_ts/imgs'
inference_folder_mask = '/work/scorreacardo_umass_edu/DeepSatGSD/data/processed/inference_ts/masks'

res_counter = []
res_dic = {
    'img_name': [],
    'year': [],
    'month': [],
    'day': [],
    'class_gsd':[],
    'iou': [],
    'se': [],
    'perc_error': [],
    'perc_area': []
}
# Iterate over the images in the inference folder
for image_name in tqdm(os.listdir(inference_folder)):
    image_path = os.path.join(inference_folder, image_name)
    mask_path = os.path.join(inference_folder_mask, image_name)

    # Load and preprocess the image
    image = np.array(Image.open(image_path)).copy()
    mask = np.array(Image.open(mask_path)).copy()
    image = image[:, :, :3]
    #mask = mask[:, :, :3]
    image = transform(image).float()
    image = image.unsqueeze(0)  # Add batch dimension
    #image = image.to(device, dtype=torch.float32)  # Move the image to GPU
    
    # Perform inference
    with torch.no_grad():
        output = model(image)
    _, predicted_class = torch.max(output, 1)
    
    predicted_class = predicted_class.item()
    res_counter.append(predicted_class)
    res_dic['img_name'].append(image_name)
    res_dic['class_gsd'].append(predicted_class)
    res_dic['year'].append(int(image_name.split("_")[0]))
    res_dic['month'].append(int(image_name.split("_")[1]))
    res_dic['day'].append(int(image_name.split("_")[2]))
    
    preprocessing_fn = smp.encoders.get_preprocessing_fn("resnet34", "imagenet")
    seg_model = get_model(class_map[predicted_class].split("_")[1], aug=True)
    seg_model.eval()
    seg_transform =  albu.Compose([
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ])

    image = imread(image_path)[:,:,:3].astype('float32')/255
    mask = imread(mask_path, as_gray=True)[:,:,np.newaxis]
    
    image_transformed = seg_transform(image=image, mask=mask)['image']
    mask_transformed = seg_transform(image=image, mask=mask)['mask']
    
    gt_mask_test = mask_transformed.squeeze()
    x_tensor_test = torch.from_numpy(image_transformed).to('cuda').unsqueeze(0)
      # Perform inference
    with torch.no_grad():
        pr_mask_test = seg_model.predict(x_tensor_test)
    pr_mask_test = (pr_mask_test.squeeze().cpu().numpy().round())

    pr_mask_test_th = sigmoid(pr_mask_test)
    pr_mask_test_th = pr_mask_test_th > 0.5
    pr_mask_test_th = np.uint8(np.multiply(pr_mask_test_th, 255))

    gt_mask_test = np.uint8(gt_mask_test)
    #calculating IoU per patch:
    iou_score = iou(pr_mask_test_th, gt_mask_test)
    se_score = se(pr_mask_test_th, gt_mask_test)
    pe_score = perc_error(pr_mask_test_th, gt_mask_test)
    pa_score = perc_area(pr_mask_test_th, gt_mask_test)
    res_dic['iou'].append(iou_score)
    res_dic['se'].append(se_score)
    res_dic['perc_error'].append(pe_score)
    res_dic['perc_area'].append(pa_score)

cuda


100%|██████████| 480/480 [01:03<00:00,  7.51it/s]
